In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import shap
from joblib import dump
from joblib import load
import seaborn as sns

In [28]:
player_df = pd.read_csv('NHL_player_season_log_all.csv')

In [29]:
for player in player_df[player_df.Tm=='TOT'].Player.unique():
    season_list = player_df[(player_df.Player == player) & (player_df.Tm=='TOT')].Season.unique()
    
    for season in season_list:
        index = player_df.index[(player_df.Player == player) & (player_df.Tm !='TOT') & (player_df.Season == season)].tolist()
        player_df.drop(index=index,inplace=True)

In [3]:
xg_reg1 = load('xgb_model1.dat') 
xg_reg2 = load('xgb_model2.dat')
xg_reg3 = load('xgb_model3.dat')

In [4]:

data1 = pd.read_csv('player_prediction_data_no_labels_1_year_back.csv')
data2 = pd.read_csv('player_prediction_data_no_labels_2_years_back.csv')
data3 = pd.read_csv('player_prediction_data_no_labels_3_years_back.csv')

In [5]:
dmatrix1 = xgb.DMatrix(data = data1[['Age', 'Player_Season_Num', 'prev_GP', 'prev_G',
       'prev_A', 'prev_PTS', 'prev_S', 'career_GP', 'career_G',
       'career_A', 'career_PTS', 'career_S', 'games', 'total_goals_per_game',
       'goals_pp', 'chances_pp', 'shots', 'shot_pct']])
dmatrix2 = xgb.DMatrix(data=data2[['Age', 'Player_Season_Num', 'prev_GP', 'prev_G',
       'prev_A', 'prev_PTS', 'prev_S', 'career_GP', 'career_G',
       'career_A', 'career_PTS', 'career_S', 'games', 'total_goals_per_game',
       'goals_pp', 'chances_pp', 'shots', 'shot_pct']])
dmatrix3 = xgb.DMatrix(data=data3[['Age', 'Player_Season_Num', 'prev_GP', 'prev_G',
       'prev_A', 'prev_PTS', 'prev_S', 'career_GP', 'career_G',
       'career_A', 'career_PTS', 'career_S', 'games', 'total_goals_per_game',
       'goals_pp', 'chances_pp', 'shots', 'shot_pct']])

In [6]:
y_pred1 = xg_reg1.predict(dmatrix1)
y_pred2 = xg_reg2.predict(dmatrix2)
y_pred3 = xg_reg3.predict(dmatrix3)

In [7]:
data1['pts_per_game_predict'] = y_pred1
data2['pts_per_game_predict'] = y_pred2
data3['pts_per_game_predict'] = y_pred3

In [8]:
data1.to_csv('Player_prediction_1_year.csv')

In [9]:
data2.to_csv('Player_prediction_2_year.csv')
data3.to_csv('Player_prediction_3_year.csv')

In [10]:
data1.columns

Index(['Unnamed: 0', 'Player', 'Season', 'Age', 'Player_Season_Num', 'PTS',
       'GP', 'prev_GP', 'prev_G', 'prev_A', 'prev_PTS', 'prev_S', 'prev_TOI',
       'career_GP', 'career_G', 'career_A', 'career_PTS', 'career_S', 'games',
       'total_goals_per_game', 'goals_pp', 'chances_pp', 'shots', 'shot_pct',
       'pts_per_game_predict'],
      dtype='object')

In [33]:
player_df = player_df[['Season', 'Age','GP','PTS','Player']]
player_df.columns

Index(['Season', 'Age', 'GP', 'PTS', 'Player'], dtype='object')

In [94]:
d1 = data1[['Player', 'Age','Season', 'pts_per_game_predict']]
d2 = data2[['Player', 'Age','Season', 'pts_per_game_predict']]
d3 = data3[['Player', 'Age','Season', 'pts_per_game_predict']]

In [30]:
player_df.shape

(29242, 29)

In [44]:
d1.shape

(29242, 4)

In [34]:
player_df.Player.nunique()

4904

In [32]:
for player in player_df.Player.unique():
    if len(player_df[player_df.Player == player]) != len(d1[d1.Player == player]):
        print(player)

In [100]:
#d1.loc[d1.Season == '2004-05','Season'] = '2005-06'
#d2.loc[d2.Season == '2004-05','Season'] = '2005-06'
#d3.loc[d3.Season == '2004-05','Season'] = '2005-06'
d1 = d1[d1.Season != '2004-05']
d2 = d2[d2.Season != '2004-05']
d3 = d3[d3.Season != '2004-05']

In [109]:
sorted(d1.Season.unique())

['1980-81',
 '1981-82',
 '1982-83',
 '1983-84',
 '1984-85',
 '1985-86',
 '1986-87',
 '1987-88',
 '1988-89',
 '1989-90',
 '1990-91',
 '1991-92',
 '1992-93',
 '1993-94',
 '1994-95',
 '1995-96',
 '1996-97',
 '1997-98',
 '1998-99',
 '1999-00',
 '2000-01',
 '2001-02',
 '2002-03',
 '2003-04',
 '2006-07',
 '2007-08',
 '2008-09',
 '2009-10',
 '2010-11',
 '2011-12',
 '2012-13',
 '2013-14',
 '2014-15',
 '2015-16',
 '2016-17',
 '2017-18',
 '2018-19',
 '2019-20',
 '2020-21']

In [101]:
df = pd.merge(player_df, d1, on=['Player','Season'], how='outer')
df['Age'] = df.apply(lambda x: x.Age_x if not x.isna().Age_x else x.Age_y, axis = 1)
df.drop(columns=['Age_x', 'Age_y'], inplace=True)

In [102]:
df = df.merge(d2, on=['Player','Season'], how='outer')
df['Age'] = df.apply(lambda x: x.Age_x if not x.isna().Age_x else x.Age_y, axis = 1)
df.drop(columns=['Age_x', 'Age_y'], inplace=True)

In [103]:
df = df.merge(d3, on=['Player','Season'], how='outer')
df['Age'] = df.apply(lambda x: x.Age_x if not x.isna().Age_x else x.Age_y, axis = 1)
df.drop(columns=['Age_x', 'Age_y'], inplace=True)

In [104]:
df['PTS_Predict_1'] = df.apply(lambda x: x.GP*x.pts_per_game_predict_x if not x.isna().GP else 82*x.pts_per_game_predict_x, axis = 1)
df['PTS_Predict_2'] = df.apply(lambda x: x.GP*x.pts_per_game_predict_y if not x.isna().GP else 82*x.pts_per_game_predict_y, axis = 1)
df['PTS_Predict_3'] = df.apply(lambda x: x.GP*x.pts_per_game_predict if not x.isna().GP else 82*x.pts_per_game_predict, axis = 1)

In [105]:
df.rename(columns={"pts_per_game_predict_x": "pts_per_game_predict_1", "pts_per_game_predict_y": "pts_per_game_predict_2","pts_per_game_predict": "pts_per_game_predict_3"},inplace=True)
df.PTS_Predict_1=df.PTS_Predict_1.apply(np.round)
df.PTS_Predict_2=df.PTS_Predict_2.apply(np.round)
df.PTS_Predict_3=df.PTS_Predict_3.apply(np.round)

In [81]:
df[df.Player == "Sidney Crosby"]

,Season,GP,PTS,Player,pts_per_game_predict_1,pts_per_game_predict_2,pts_per_game_predict_3,Age,PTS_Predict_1,PTS_Predict_2,PTS_Predict_3
5420,2005-06,81.0,102.0,Sidney Crosby,NaN,NaN,NaN,18.0,NaN,NaN,NaN
5421,2006-07,79.0,120.0,Sidney Crosby,1.260733,NaN,NaN,19.0,100.0,NaN,NaN
5422,2007-08,53.0,72.0,Sidney Crosby,1.345005,1.161697,NaN,20.0,71.0,62.0,NaN
5423,2008-09,77.0,103.0,Sidney Crosby,1.340203,1.256599,1.271685,21.0,103.0,97.0,98.0
5424,2009-10,81.0,109.0,Sidney Crosby,1.381964,1.246673,1.374489,22.0,112.0,101.0,111.0
5425,2010-11,41.0,66.0,Sidney Crosby,1.324251,1.440349,1.399006,23.0,54.0,59.0,57.0
5426,2011-12,22.0,37.0,Sidney Crosby,1.432071,1.384025,1.450086,24.0,32.0,30.0,32.0
5427,2012-13,36.0,56.0,Sidney Crosby,1.477455,1.348515,1.308170,25.0,53.0,49.0,47.0
5428,2013-14,80.0,104.0,Sidney Crosby,1.250582,1.248537,1.151145,26.0,100.0,100.0,92.0
5429,2014-15,77.0,84.0,Sidney Crosby,1.135197,1.120528,1.104619,27.0,87.0,86.0,85.0


In [114]:
df.to_csv('NHL_Player_Points_Predictions.csv')

In [110]:
tst_df = df[df.Season=="2005-06"]

In [113]:
tst_df[tst_df.Player == "Teemu Selanne"]

,Season,GP,PTS,Player,pts_per_game_predict_1,pts_per_game_predict_2,pts_per_game_predict_3,Age,PTS_Predict_1,PTS_Predict_2,PTS_Predict_3
23706,2005-06,80.0,90.0,Teemu Selanne,NaN,0.711149,0.853805,35.0,NaN,57.0,68.0


In [112]:
for player in tst_df.Player:
    if len(tst_df[tst_df.Player == player])>1:
        print(player)